In [3]:
from gs_quant.risk import CarryScenario, MarketDataPattern, MarketDataShock, MarketDataShockBasedScenario,MarketDataShockType
from gs_quant.session import Environment, GsSession
from gs_quant.common import PayReceive, Currency
from gs_quant.instrument import IRSwap

In [4]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [5]:
# define swap
swap = IRSwap(termination_date='10y', notional_currency=Currency.USD, pay_or_receive=PayReceive.Pay, fixed_rate=0.007)

In [6]:
# 1 bp spot shock
ir_1bp_scenario = MarketDataShockBasedScenario(shocks={
    MarketDataPattern('IR', 'USD') : MarketDataShock(shock_type=MarketDataShockType.Absolute, value=1e-4),
    MarketDataPattern('IR Reset', 'USD-3m') : MarketDataShock(shock_type=MarketDataShockType.Absolute, value=1e-4)
})

# 22d carry shock
carry_22b_scenario = CarryScenario(date='22b')

In [8]:
# scenario carrying swap 22d forward and then applying 1bp spot shock
with carry_22b_scenario, ir_1bp_scenario:
    carry_22b_then_ir_1bp_price = swap.price()
    
# Look at the difference between scenario and base prices
print('Base price:     {:,.2f}'.format(swap.price()))
print('Scenario price: {:,.2f}'.format(carry_22b_then_ir_1bp_price))